In [17]:
batch_size=25
num_batches=4
num_records=1000
## Each task handles 25*4 records, there should be 10 partitions overall to process 1000 records.

def getBatchSentiment(input_list):
    j=[i[1] for i in input_list]
    client = boto3.client('comprehend',region_name='us-west-2')
    for i in range(num_batches-1):
       k=j[batch_size*i:batch_size*(i+1)]
       response = client.batch_detect_sentiment(
          TextList=k,
          LanguageCode='en')
       for r in response['ResultList']:
            print (r['Sentiment'])
            input_list[r['Index']]=(input_list[r['Index']][0],input_list[r['Index']][1],r['Sentiment'])
    return input_list

In [3]:
import random, boto3
reviews=['I like this product a lot','Too expensive. A waste of money']

items=[]
for i in range(num_records):
    items.append((i,random.choice(reviews),''))
print (items)

[(0, 'I like this product a lot', ''), (1, 'Too expensive. A waste of money', ''), (2, 'I like this product a lot', ''), (3, 'I like this product a lot', ''), (4, 'I like this product a lot', ''), (5, 'Too expensive. A waste of money', ''), (6, 'Too expensive. A waste of money', ''), (7, 'Too expensive. A waste of money', ''), (8, 'I like this product a lot', ''), (9, 'Too expensive. A waste of money', ''), (10, 'Too expensive. A waste of money', ''), (11, 'Too expensive. A waste of money', ''), (12, 'I like this product a lot', ''), (13, 'I like this product a lot', ''), (14, 'I like this product a lot', ''), (15, 'I like this product a lot', ''), (16, 'I like this product a lot', ''), (17, 'I like this product a lot', ''), (18, 'Too expensive. A waste of money', ''), (19, 'I like this product a lot', ''), (20, 'Too expensive. A waste of money', ''), (21, 'Too expensive. A waste of money', ''), (22, 'Too expensive. A waste of money', ''), (23, 'I like this product a lot', ''), (24, 'T

In [4]:
rdd=sc.parallelize(items)
df=rdd.toDF(['review_id','review_text','sentiment'])

In [5]:
df.show()
df.cache()

+---------+--------------------+---------+
|review_id|         review_text|sentiment|
+---------+--------------------+---------+
|        0|I like this produ...|         |
|        1|Too expensive. A ...|         |
|        2|I like this produ...|         |
|        3|I like this produ...|         |
|        4|I like this produ...|         |
|        5|Too expensive. A ...|         |
|        6|Too expensive. A ...|         |
|        7|Too expensive. A ...|         |
|        8|I like this produ...|         |
|        9|Too expensive. A ...|         |
|       10|Too expensive. A ...|         |
|       11|Too expensive. A ...|         |
|       12|I like this produ...|         |
|       13|I like this produ...|         |
|       14|I like this produ...|         |
|       15|I like this produ...|         |
|       16|I like this produ...|         |
|       17|I like this produ...|         |
|       18|Too expensive. A ...|         |
|       19|I like this produ...|         |
+---------+

In [13]:
record_count=df.count()
print record_count
df1=df.repartition(record_count/(num_batches*batch_size))
df1.rdd.getNumPartitions()

1000
10

In [18]:
rdd1=df1.sortWithinPartitions(['review_id'],ascending=True).rdd
rdd2=rdd1.map(lambda line: tuple([str(x) for x in line])).glom()
output=rdd2.map(lambda l: getBatchSentiment(l)).flatMap(lambda x:x).collect()
print (output)

[('9', 'Too expensive. A waste of money', u'NEGATIVE'), ('19', 'I like this product a lot', u'POSITIVE'), ('29', 'I like this product a lot', u'POSITIVE'), ('39', 'I like this product a lot', u'NEGATIVE'), ('49', 'I like this product a lot', u'POSITIVE'), ('59', 'Too expensive. A waste of money', u'POSITIVE'), ('69', 'I like this product a lot', u'NEGATIVE'), ('79', 'Too expensive. A waste of money', u'NEGATIVE'), ('89', 'I like this product a lot', u'NEGATIVE'), ('99', 'Too expensive. A waste of money', u'POSITIVE'), ('109', 'Too expensive. A waste of money', u'NEGATIVE'), ('119', 'Too expensive. A waste of money', u'NEGATIVE'), ('129', 'Too expensive. A waste of money', u'POSITIVE'), ('139', 'I like this product a lot', u'POSITIVE'), ('149', 'Too expensive. A waste of money', u'POSITIVE'), ('159', 'Too expensive. A waste of money', u'POSITIVE'), ('169', 'Too expensive. A waste of money', u'POSITIVE'), ('179', 'I like this product a lot', u'NEGATIVE'), ('189', 'I like this product a l

In [16]:
len(output)

1000